# 🛡️ Chunking and tokenizing HTML documents using Data Prep Kit and the Docling Transforms

This notebook demonstrates how to build a sequence of <a href=https://github.com/data-prep-kit/data-prep-kit> <b>DPK transforms</b> </a> for ingesting HTML documents using Docling2Parquet transforms and chunking them using Doc_Chunk transform. Both transforms are based on the <a href=https://docling-project.github.io/docling/> Docling library</a>. 

In this example, we will use the <i>Wikimedia API<i> to retrieve the HTML articles that will be used as a seed for our LLM application. Once the articles are loaded to a local cache, we will construct and invoke the sequence of transforms to ingest the content and produce the embedding for the chuncked content.


## 🔍 Why DPK Pipelines

DPK transform pipelines are intended to simplify how any number of transforms can be executed in a sequence to ingest, annotate, filter and create embedding used for LLM post-training and RAG applications. 


## 🧰 Key Transforms in This Recipe

We will use the following transforms from DPK:

- `Docling2Parquet`: Ingest one or more HTML document and turn it into a parquet file.
- `Doc_Chunk`: Create chunks from one more more ducment.
- `Tokenization`: Create embedding for document chunks.


## Prerequisites

1- This notebook uses Wikimedia API for retrieving the initial HTML documents and llama-tokenizer from hugging face. 

2- In order to use the notebook, users must provide a <b>.env</b> file with a valid access tokens to be used for accessing the wikimedia endpoint (<a href=https://enterprise.wikimedia.com/docs/> instructions can be found here </a>) and a Hugging face token for loading the model (<a href=https://huggingface.co/docs/hub/en/security-tokens> instructions can be found here</a>). The .env file will look something like this:
```
WIKI_ACCESS_TOKEN='eyxxx'
HF_READ_ACCESS_TOKEN='hf_xxx'
```

3- Install DPK library to environment

In [ ]:
%%capture
%pip install "data-prep-toolkit-transforms[docling2parquet,doc_chunk,tokenization]"
%pip install pandas
%pip install "numpy<2.0"
from dotenv import load_dotenv

load_dotenv(".env", override=True)

We will define and use a utility function for downloading the articles and saving them to the local disk:

<b>load_corpus</b>: Uses http request with the wikimedia api token to connect to a Wikimedia endpoint and retrieve the HTML articles that will be used as a seed for our LLM application. The article will then be saved to a local cache folder for further processing


In [ ]:
def load_corpus(articles: list, folder: str) -> int:
    import os
    import re

    import requests

    headers = {"Authorization": f"Bearer {os.getenv('WIKI_ACCESS_TOKEN')}"}
    count = 0
    for article in articles:
        try:
            endpoint = f"https://api.enterprise.wikimedia.com/v2/articles/{article}"
            response = requests.get(endpoint, headers=headers)
            response.raise_for_status()
            doc = response.json()
            for article in doc:
                filename = re.sub(r"[^a-zA-Z0-9_]", "_", article["name"])
                with open(f"{folder}/{filename}.html", "w") as f:
                    f.write(article["article_body"]["html"])
                    count = count + 1
        except Exception as e:
            print(f"Failed to retrieve content: {e}")
    return count

## 🔗 Setup the experiment

DPK requires that we define a source/input folder where the transform sequence will be ingesting the document and a destination/output folder where the embedding will be stored. We will also initialize the list of articles we want to use in our application


In [ ]:
import os
import tempfile

datafolder = tempfile.mkdtemp(dir=os.getcwd())
articles = ["Science,_technology,_engineering,_and_mathematics"]
assert load_corpus(articles, datafolder) > 0, "Faild to download any documents"

### 🔗 Injest

Invoke Docling2Parquet tansform that will parse the HTML document and create a Markdown

In [ ]:
%%capture
from dpk_docling2parquet import Docling2Parquet, docling2parquet_contents_types

result = Docling2Parquet(
    input_folder=datafolder,
    output_folder=f"{datafolder}/docling2parquet",
    data_files_to_use=[".html"],
    docling2parquet_contents_type=docling2parquet_contents_types.MARKDOWN,  # markdown
).transform()

### 🔗 Chunk

Invoke DocChunk tansform to break the HTML document into chunks

In [ ]:
%%capture
from dpk_doc_chunk import DocChunk

result = DocChunk(
    input_folder=f"{datafolder}/docling2parquet",
    output_folder=f"{datafolder}/doc_chunk",
    doc_chunk_chunking_type="li_markdown",
    doc_chunk_chunk_size_tokens=128,  # default 128
    doc_chunk_chunk_overlap_tokens=30,  # default 30
).transform()

### 🔗 Tokenization

Invoke Tokenization transform to create embedding of various chunks

In [ ]:
%%capture
from dpk_tokenization import Tokenization

Tokenization(
    input_folder=f"{datafolder}/doc_chunk",
    output_folder=f"{datafolder}/tkn",
    tkn_tokenizer="hf-internal-testing/llama-tokenizer",
    tkn_chunk_size=20_000,
).transform()

## ✅ Summary

This notebook demonstrated how to run a DPK pipeline using IBM's Data Prep Kit and the Docling library. Each transform create one or more parquet files that users can explore to better understand what each stage of the pipeline produces. 
The see the output of the final stage, we will use Pandas to read the final parquet file and display its content

In [ ]:
from pathlib import Path

import pandas as pd

parquet_files = list(Path(f"{datafolder}/tkn/").glob("*.parquet"))
pd.concat(pd.read_parquet(file) for file in parquet_files)

,tokens,document_id,document_length,token_count
0,"[1, 444, 11814, 262, 3002]",f1f5b56a78829ab2165b3bbeb94b1167e4c5583c437f1d...,14,5
1,"[1, 835, 5298, 13, 13, 797, 278, 4688, 29871, ...",402e82a9e81cc3d2494fac36bebf8bf1a2662800e5a00c...,2100,655
2,"[1, 835, 5901, 21833, 13, 13, 29899, 321, 1254...",4fb389d0f0e999c2496f137b4a7c0671e79c09cf9477e9...,2833,968
3,"[1, 444, 26304, 4978, 13, 13, 14136, 1967, 666...",3709997548d84224361a6835760b5ae48a1637e78d54a0...,1496,483
4,"[1, 444, 2648, 4234]",1e1a58ad5664d963bc207dc791825258c33337c2559f6a...,13,4
5,"[1, 835, 8314, 13, 13, 1576, 9870, 315, 1038, ...",83a63864e5ddfdd41ef0f813fb7aa3c95e04c029c32ab3...,1340,442
6,"[1, 835, 7400, 13, 13, 6028, 1114, 27871, 2987...",5e29fb4e4cf37ed4c49994620e4a00da9693bc061e82c1...,1800,548
7,"[1, 835, 7551, 13, 13, 25411, 3762, 8950, 6020...",3fc34013d93391a7504e84069190479fbc85ba7e7072cb...,1784,511
8,"[1, 835, 4092, 13, 13, 13393, 884, 29901, 518,...",e8b28e20e3fc3da40b6b368e30f9c953f5218370ec2f7a...,774,229
9,"[1, 3191, 18312, 13, 13, 1576, 365, 29965, 152...",94b54fbda274536622f70442b18126f554610e8915b235...,1076,263
